#### Text Decection with Tesseract OCR

Import Libraries

In [1]:
import pytesseract

Check if Tesseract is installed

In [2]:
pytesseract.__version__

'0.3.9'

In [3]:

pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

Import image

In [29]:
IMAGE_PATH = "../sample_data/german_sample.jpeg"

Import CV2

In [4]:
import cv2

In [5]:
print(cv2.getBuildInformation())


General configuration for OpenCV 4.5.4 =====================================
  Version control:               4.5.4

  Platform:
    Timestamp:                   2021-11-19T18:52:46Z
    Host:                        Windows 10.0.17763 AMD64
    CMake:                       3.22.0
    CMake generator:             Visual Studio 14 2015
    CMake build tool:            MSBuild.exe
    MSVC:                        1900
    Configuration:               Debug Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (15 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (4 files):             + SSSE3 SSE4_1 POPCNT SSE4_2 AVX
      AVX2 (30 files):           + SSSE3 SSE4_1 

Read image

In [30]:
img = cv2.imread(IMAGE_PATH)

Get Grayscale image

In [31]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img

array([[157, 155, 155, ...,  37,  34,  35],
       [158, 156, 156, ...,  37,  38,  42],
       [159, 158, 157, ...,  36,  39,  45],
       ...,
       [ 89,  90,  91, ...,  67,  69,  76],
       [ 90,  90,  89, ...,  68,  69,  76],
       [ 91,  89,  88, ...,  73,  74,  81]], dtype=uint8)

Remove Noise

In [12]:
img = cv2.medianBlur(img, 5)
img

array([[157, 157, 157, ...,  39,  37,  36],
       [157, 157, 157, ...,  39,  38,  37],
       [157, 157, 157, ...,  39,  39,  37],
       ...,
       [ 90,  90,  90, ...,  74,  75,  76],
       [ 90,  90,  90, ...,  74,  76,  76],
       [ 90,  90,  90, ...,  76,  76,  76]], dtype=uint8)

In [13]:
img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
img

array([[255, 255, 255, ...,   0,   0,   0],
       [255, 255, 255, ...,   0,   0,   0],
       [255, 255, 255, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)

Opening - erosion followed by dilation

In [8]:
import numpy as np

In [16]:
kernel = np.ones((5,5),np.uint8)
img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
img

array([[255, 255, 255, ...,   0,   0,   0],
       [255, 255, 255, ...,   0,   0,   0],
       [255, 255, 255, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)

Canny Edge Detection

In [18]:
img = cv2.Canny(img, 100, 200)
img

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

Detect Text with OCR

In [35]:
text = pytesseract.image_to_string(img).split("\n")
text

['THOMY.',
 '',
 '~ SEIT 1954 =',
 '',
 'LES SAUCES',
 '',
 'HOLLANDAISE',
 '',
 'LAKTOSEFRE!',
 'win a',
 '',
 'i',
 '4',
 '',
 'Ns',
 '',
 'NUTRI-SCORE',
 '']

In [6]:
from pytesseract import Output
import re

In [7]:
img = cv2.imread('../sample_data/invoice-sample.jpg')
d = pytesseract.image_to_data(img, output_type=Output.DICT)
keys = list(d.keys())

date_pattern = '^(0[1-9]|[12][0-9]|3[01])/(0[1-9]|1[012])/(19|20)\d\d$'

n_boxes = len(d['text'])

for i in range(n_boxes):
    if int(float(d['conf'][i])) > 60:
        if re.match(date_pattern, d['text'][i]):
            print(d['text'][i])
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imshow('img', img)
cv2.waitKey(0)









12/12/2001


error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1274: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
